In [11]:
import json
import os
import pandas as pd
from difflib import SequenceMatcher
import jellyfish
from geopy.geocoders import Nominatim
geolocator = Nominatim()
from geopy.distance import great_circle
import folium
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap

In [12]:
#Center coordinates
SF_COORDINATES = (40.4168, -3.7038)

# for speed purposes
MAX_RECORDS = 10

In [13]:
os.chdir('/Volumes/MacintoshHD/_GitHub/doctordata/api/data/')
os.listdir()

['.DS_Store',
 '.~lock.streetlamp_missing.csv#',
 '20171110_InventarioFuentes.csv',
 'anexo i monumentos.xls',
 'artwork.geojson',
 'drinking_water.geojson',
 'indice.csv',
 'memorial.geojson',
 'monument.geojson',
 'street_lamp.geojson']

In [14]:
OSM_fichero = 'street_lamp.geojson'
#AYU_fichero = '20171110_InventarioFuentes.csv'

In [15]:
with open(OSM_fichero) as f:
    OSM = json.load(f)
f.close()

In [16]:
ids = [OSM['features'][i]['id'] for i in range(len(OSM['features']))]
lat = [OSM['features'][i]['geometry']['coordinates'][1] for i in range(len(OSM['features']))]
lon = [OSM['features'][i]['geometry']['coordinates'][0] for i in range(len(OSM['features']))]
#name = [OSM['features'][i]['properties']['name'] for i in range(len(OSM['features']))]
ayu = ids[:]

In [17]:
AYU_MISSING = []
for i in range(len(ids)):
    if isinstance(ayu[i], str):
        AYU_MISSING.append((ids[i],lat[i],lon[i]))
        
AYU_MISSING = pd.DataFrame(AYU_MISSING)
AYU_MISSING.columns = ['id_OSM','LATITUD','LONGITUD']

AYU_MISSING = AYU_MISSING.set_index('id_OSM')
AYU_MISSING['coord'] = list(zip(AYU_MISSING.LATITUD.values, AYU_MISSING.LONGITUD.values))

In [22]:
os.chdir('/Volumes/MacintoshHD/_GitHub/doctordata/api/csv/')
AYU_MISSING.to_csv(OSM_fichero[:-8]+'-missing.csv')

In [19]:
os.chdir('/Volumes/MacintoshHD/_GitHub/doctordata/api/web/')

In [20]:
map = folium.Map(location=SF_COORDINATES, zoom_start=11)
marker_cluster = MarkerCluster().add_to(map)
for i in range(len(AYU_MISSING.index.values)):
    folium.Marker(icon=folium.Icon(color='green'),popup=folium.Popup(AYU_MISSING.index.values[i], parse_html=True),
        location = (AYU_MISSING.LATITUD.values[i],AYU_MISSING.LONGITUD.values[i])).add_to(marker_cluster)
map.save(OSM_fichero[:-8]+'-missing_Ayu.html')

In [21]:
map = folium.Map(location=SF_COORDINATES, zoom_start=11)
HeatMap(AYU_MISSING['coord'].values).add_to(map)
map.save(OSM_fichero[:-8]+'-missing_Ayu_heat.html')

In [ ]:
map